In [1]:
from skimage import io, measure
import matplotlib.pyplot as plt
import os
import numpy as np 
import napari
import h5py
from tqdm import tqdm
import statistics
import scipy.ndimage as ndi
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
from pathlib import Path

# Read data

In [2]:
im_folder = "/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/tifs"
im_folder = "/Users/thomasathey/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/jellyfish-imaging/25_01_07tommy_ct_2025-01-06_170445/tiffs"

res = 1.382273

res_3d = (res, res, res)

## Collect 2d images from im_folder

In [3]:
# identify all tifs in im_folder and sort them by their name
im_files = os.listdir(im_folder)
im_files = [f for f in im_files if f.endswith('.tiff')]
im_files.sort()

# read all tifs and stack them into a 3D array
im_stack = []
for f in tqdm(im_files):
    im = io.imread(os.path.join(im_folder, f))
    im_stack.append(im)
im_stack = np.array(im_stack)

image = im_stack

im_stack.shape

  0%|          | 0/993 [00:00<?, ?it/s]

(993, 1013, 990)

## Read a 3d image

In [ ]:
image = io.imread("/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/3d_processed_files/im_full.tif")

print(f"Image shape: {image.shape}")

In [3]:
seg_obj = io.imread("/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/3d_processed_files/obj_full.tif")
seg_subum = io.imread("/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/3d_processed_files/subum_full.tif")

In [4]:
folder = Path("/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/3d_processed_files")

image_crop = io.imread(folder / "im_hemi.tif")
seg_subum_crop = io.imread(folder / "subum_hemi.tif")
seg_obj_crop = io.imread(folder / "obj_hemi.tif")

## Read new 3d image

In [2]:
image = io.imread("/Users/thomasathey/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/jellyfish-imaging/25_01_07tommy_ct_2025-01-06_170445/derivatives/im_full.tif")

print(f"Image shape: {image.shape}")

Image shape: (993, 1013, 990)


## View data in napari

### Image only in unit resolution

In [8]:
io.imsave("/Users/thomasathey/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/jellyfish-imaging/25_01_07tommy_ct_2025-01-06_170445/derivatives/im_full.tif", image)

In [5]:
image_crop_rough = image#[:940,:,:] #image[:940,200:,:]

In [6]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image_crop_rough)

<Image layer 'image_crop_rough' at 0x1466c38c0>

In [5]:


np.linalg.norm(np.subtract(b,c))


np.float64(497.51884386422995)

In [19]:
points = np.array([a,b,c,d])
mx = np.amax(points)
im_blank = 100*np.ones((mx, mx, mx), dtype=np.uint8)
im_blank[0,0,0] = 0

viewer = napari.Viewer()
viewer.add_image(im_blank)
viewer.add_points(points[:1,:], size=10, face_color='red', border_color='white')
viewer.add_points(points[1:,:], size=10, face_color='green', border_color='white')

<Points layer 'Points [1]' at 0x1558d7c20>

### True resolution

In [8]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image, scale=res_3d)
viewer.add_labels(seg_obj, scale=res_3d)
viewer.add_labels(seg_subum, scale=res_3d)
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

In [6]:
viewer = napari.Viewer()

In [13]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image_crop, scale=res_3d)
viewer.add_labels(seg_obj_crop, scale=res_3d)
viewer.add_labels(seg_subum_crop, scale=res_3d)
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

In [15]:
p1 = (973,1026)
p2 = (1112, 1140)

np.linalg.norm(np.subtract(p2,p1)*res)

np.float64(248.49024491894588)

# Gonad coords

In [3]:
# zeiss viewer
a = [496, 712, 788]
b = [356,410, 721]
c = [674, 686, 456]
d = [456, 388, 373]

mouth = [563, 564, 467]

# reverse into napari coordinates
a.reverse()
b.reverse()
c.reverse()
d.reverse()

points = np.array([a,b,c,d])

In [4]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image_crop_rough)
viewer.add_points([a], size=10, face_color='red', border_color='white')
viewer.add_points([b,c], size=10, face_color='orange', border_color='white')
viewer.add_points([d], size=10, face_color='green', border_color='white')
viewer.add_points([mouth], size=10, face_color='blue', border_color='white')

NameError: name 'image_crop_rough' is not defined

## Normal vector

In [5]:
v1 = np.subtract(b,c)
v2 = np.subtract(b,mouth)

v1 = v1.astype(float)
v2 = v2.astype(float)

# v1 /= np.linalg.norm(v1)
# v2 /= np.linalg.norm(v2)

normal = np.cross(v1,v2)
normal /= np.linalg.norm(normal)

normal2 = (normal[-1], normal[1], normal[2])

In [6]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image)
viewer.add_points([a], size=10, face_color='red', border_color='white')
viewer.add_points([b,c], size=10, face_color='orange', border_color='white')
viewer.add_points([d], size=10, face_color='green', border_color='white')
viewer.add_points([mouth], size=10, face_color='blue', border_color='white')



plane_parameters = {
    'position': (0, 0, 0),
    'normal': normal
    }
plane_layer = viewer.add_image(
    image,
    name='plane',
    rendering='average',
    depiction='plane',
    blending='additive',
    colormap='cyan',
    plane=plane_parameters
    )
